In [2]:
import os
import shutil
import sys
import argparse
from pathlib import Path
import idd_climate_models.constants as rfc
from idd_climate_models.tc_risk_functions import create_custom_namelist, execute_tc_risk



In [3]:
data_source = 'cmip6'
model = 'ACCESS-CM2'
variant = 'r1i1p1f1'
scenario = 'ssp126'
time_bin = '2015-2034'
basin = 'GL'
args = argparse.Namespace(
    data_source=data_source,
    model=model,
    variant=variant,
    scenario=scenario,
    time_bin=time_bin,
    basin=basin
)
create_custom_namelist(args)
execute_tc_risk(args, script_name='compute')

✅ Configuration successfully created at: /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/output/cmip6/ACCESS-CM2/r1i1p1f1/ssp126/2015-2034/GL/namelist.py
Generating land masks...
Computing monthly mean and variance of environmental wind...
Saved /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/output/cmip6/ACCESS-CM2/r1i1p1f1/ssp126/2015-2034/env_wnd_ACCESS-CM2_ssp126_r1i1p1f1_201501_203412.nc
Time Elapsed: 972.319533 s
Computing thermodynamic variables...
Detected monthly time steps; setting day to 15th of month.
Saved /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/output/cmip6/ACCESS-CM2/r1i1p1f1/ssp126/2015-2034/thermo_ACCESS-CM2_ssp126_r1i1p1f1_201501_203412.nc
Time Elapsed: 91.017792 s
✅ Run complete.


In [ ]:

import idd_climate_models.tc_risk_config_defaults as tc_risk_defaults
# 1) Use constants for external repo path
TC_RISK_REPO_ROOT_DIR = rfc.TC_RISK_REPO_ROOT_DIR

# --- Configuration Generation Functions ---
# Your project's internal structure for the reference config
repo_name = rfc.repo_name
package_name = rfc.package_name
# Assuming 'namelist_config_original.py' is the reference file
REFERENCE_CONFIG_PATH = rfc.REPO_ROOT / repo_name / "src" / package_name / "01_run_tc_risk" / 'namelist_config_original.py' 


In [ ]:
# --- Main Orchestration ---

def main():
    parser = argparse.ArgumentParser(
        description="Generates a custom namelist and executes the TC-risk model using climate model inputs.",
        formatter_class=argparse.RawTextHelpFormatter
    )

    # --- New Input-Defining Arguments (from the second script) ---
    parser.add_argument('--data-source', type=str, default='cmip6',
                        help="Data source (e.g., cmip6).")
    parser.add_argument('--model', type=str, default='ACCESS-CM2',
                        help="Climate model name.")
    parser.add_argument('--variant', type=str, default='r1i1p1f1',
                        help="Model variant/ensemble member.")
    parser.add_argument('--scenario', type=str, default='historical',
                        help="Scenario/experiment name.")
    parser.add_argument('--time-bin', type=str, default='1970-1989',
                        help="Time period (e.g., 1970-1989). This sets start-year and end-year.")

    parser.add_argument(
        '-s', '--script-type', type=str, required=True,
        choices=['compute', 'compute_to_var_cov'],
        help="The specific TC-risk script function to execute."
    )
    parser.add_argument(
        '-b', '--basin', type=str, required=True,
        help="The basin identifier to pass to the execution function (e.g., NA, WP)."
    )
       
    args = parser.parse_args()
    arg_dict = vars(args)

    # 1. Derive Custom Namelist Parameters from Input Arguments
    
    # 1A. Determine Base and Output Paths

        
    

    

    # 1B. Extract Start and End Year from time_bin

    
    # 2. Build Replacements Dictionary (Automatic and Command-Line)
     # Assuming 'end_year' is also a variable in the namelist

    # (3) Update the namelist with optional command-line overrides
    # Process only non-derived, optional arguments provided by the user
    for arg_name, value in arg_dict.items():
        if value is not None and arg_name not in ['data_source', 'model', 'variant', 'scenario', 'time_bin', 'script_type', 'basin']:
            # The variable name in the namelist file must match the argument name (e.g., 'n_procs')
            var_name = arg_name.replace('-', '_') 
            
            if isinstance(value, str):
                formatted_value = f"'{value}'"
            else:
                formatted_value = str(value)
            
            replacements[var_name] = formatted_value

    print(f"Namelist replacements: {replacements}")

    # 3. Create and Save Custom Namelist
    generated_namelist_path = modify_and_save_config(
        original_path=REFERENCE_CONFIG_PATH,
        new_path=custom_namelist_path,
        replacements=replacements
    )

    if generated_namelist_path:
        # 4. Execute TC-Risk Model
        tc_risk_repo_path = Path(TC_RISK_REPO_ROOT_DIR).expanduser().resolve()
        
        execute_tc_risk(
            tc_risk_repo_path=tc_risk_repo_path,
            custom_namelist_path=generated_namelist_path,
            basin_name=args.basin,
            script_name=args.script_type
        )


if __name__ == '__main__':
    main()

In [ ]:
# Example 1: Run 'compute' script (first run)
python run_tc_risk_pipeline.py \
    --config-name 'run_A_compute' \
    --script-type 'compute' \
    --basin 'NA' \
    --output-root '/mnt/results/my_project_output' \
    --exp-name 'NA_baseline' \
    --start-year 2075 \
    --n-procs 8

1296000

In [ ]:
# Example 2: Run 'compute_to_var_cov' script (second run)
python run_tc_risk_pipeline.py \
    --config-name 'run_B_varcov_scenario1' \
    --script-type 'compute_to_var_cov' \
    --basin 'WP' \
    --output-root '/mnt/results/my_project_output' \
    --exp-name 'WP_scenario1' \
    --tracks-per-year 50 \
    --n-procs 16

# Example 3: Third run
python run_tc_risk_pipeline.py \
    --config-name 'run_C_varcov_scenario2' \
    --script-type 'compute_to_var_cov' \
    --basin 'WP' \
    --output-root '/mnt/results/my_project_output' \
    --exp-name 'WP_scenario2' \
    --tracks-per-year 50 \
    --n-procs 16
# ... and so on for the remaining 5 runs ...

In [ ]:
import importlib
import sys
import getpass
import uuid
import os
from jobmon.client.tool import Tool 
from pathlib import Path

# NOTE: These imports rely on external module definitions (constants, io_compare_utils, etc.)
import idd_climate_models.constants as rfc
from idd_climate_models.validation_functions import validate_all_models_in_source, create_validation_dict
from idd_climate_models.dictionary_utils import get_models
from idd_climate_models.dictionary_utils import parse_results


# --- CONSTANT DEFINITIONS (from rfc) ---
repo_name = rfc.repo_name
package_name = rfc.package_name
TC_RISK_INPUT_PATH = rfc.TC_RISK_INPUT_PATH
TC_RISK_OUTPUT_PATH = rfc.TC_RISK_OUTPUT_PATH
SCRIPT_ROOT = rfc.REPO_ROOT / repo_name / "src" / package_name / "01_run_tc_risk"

# Configuration
DATA_SOURCE = "cmip6"
NUM_DRAWS = 20

INPUT_DATA_TYPE = "tc_risk"
INPUT_IO_TYPE = "input"
OUTPUT_DATA_TYPE = "tc_risk"
OUTPUT_IO_TYPE = "output"



# ============================================================================
# DATA SETUP & VALIDATION
# ============================================================================

input_validation_dict = create_validation_dict(
    INPUT_DATA_TYPE, 
    INPUT_IO_TYPE, 
    DATA_SOURCE,
    strict_grid_check=False
)
input_validation_dict = validate_all_models_in_source(
    validation_dict=input_validation_dict,
    verbose=False,
    strict_grid_check=False # <-- CRITICAL: PASS FLAG to function
)
input_complete_models = get_models(input_validation_dict, complete_only=True)

models_to_process = input_complete_models




Validation complete for: tc_risk, input, cmip6
Summary: 22/22 models complete. Parsed log (up to 'scenario') written to /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/input/cmip6/validation_log.json


TypeError: 'set' object is not subscriptable

In [5]:
validation_results_for_tasks = parse_results(
    input_validation_dict, 
    detail='time-period'
)

In [7]:
import xarray as xr
from pathlib import Path

# Set the path to the required file (ta or hus, since they contain plev)
file_path = Path("/mnt/team/rapidresponse/pub/tropical-storms/tc_risk/input/cmip6/ACCESS-CM2/r1i1p1f1/historical/1970-1989/ta_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_197001-198912.nc")

# Open the dataset
try:
    ds = xr.open_dataset(file_path)
    print("✅ File opened successfully.")
except Exception as e:
    print(f"❌ Error opening file: {e}")

✅ File opened successfully.


In [8]:
plev_coord = ds['plev']

In [10]:
# Assuming the pressure coordinate is named 'plev'
plev_coord = ds['plev'] 

# 1. Check Units
print("\n--- UNITS ---")
if 'units' in plev_coord.attrs:
    print(f"The 'plev' coordinate units are: {plev_coord.attrs['units']}")
else:
    print("WARNING: 'plev' units attribute not found.")

# 2. Check Values
print("\n--- FIRST 5 VALUES ---")
print(plev_coord[:5].values)

print("\n--- LAST 5 VALUES ---")
print(plev_coord[-5:].values)

# 3. Check for specific 250 hPa value (25000 Pa)
target_pa = 25000.0

if target_pa in plev_coord.values:
    print(f"\nTarget value {target_pa} is PRESENT in plev values.")
else:
    print(f"\nTarget value {target_pa} is MISSING from plev values.")


--- UNITS ---
The 'plev' coordinate units are: Pa

--- FIRST 5 VALUES ---
[100000.00000001  92500.00000001  85000.00000001  70000.00000001
  60000.00000001]

--- LAST 5 VALUES ---
[3000.00000001 2000.00000001 1000.00000001  500.00000001  100.00000001]

Target value 25000.0 is MISSING from plev values.


In [2]:
import xarray as xr
path = '/mnt/team/rapidresponse/pub/tropical-storms/tc_risk/input/cmip6/FGOALS-g3/r1i1p1f1/ssp245/2035-2054/ua_day_FGOALS-g3_ssp245_r1i1p1f1_gn_20350101-20541231.nc'
ds = xr.open_dataset(path)

In [3]:
ds

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 7300, bnds: 2, lat: 80, lon: 180, plev: 8)
Coordinates:
  * time       (time) object 58kB 2035-01-01 12:00:00 ... 2054-12-31 12:00:00
  * lat        (lat) float64 640B -90.0 -84.82 -80.72 ... 80.72 84.82 90.0
  * lon        (lon) float64 1kB 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * plev       (plev) float64 64B 1e+05 8.5e+04 7e+04 ... 1e+04 5e+03 1e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object 117kB ...
    lat_bnds   (time, lat, bnds) float64 9MB ...
    lon_bnds   (time, lon, bnds) float64 21MB ...
    ua         (time, plev, lat, lon) float32 3GB ...
Attributes: (12/46)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  60225.0
    contact:                Lijuan Li (ljli@mail.iap.ac.cn)
    ...                     ...
    title:                  FGOALS-g3 output prepared for CMIP6
    tracking_id:            hdl:21.14100/61511a3e-fafd-4bcd-9892-a969e7b53a37
    variable_id:            ua
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by Lawrence Livermore P...
    cmor_version:           3.5.0